# Objetivo

El objetivo de este notebook es recolectar información de las publicaciones del site de MercadoLibre para luego realizar un modelado simple para predecir, dada una publicación, la cantidad de items vendidos. (El modelado se lleva a cabo en el notebook "modelado").

La información se recolecta a través de las APIS publicas de MercadoLibre (https://developers.mercadolibre.com.ar/es_ar/api-docs-es).

Es importante notar que se tomó en cuenta sólo el site de Argentina y sólo las publicaciones bajo la categoría "Celulares y Teléfonos". Se tomaron 1000 publicaciones por cada una de las subcategorías de la categoría "Celulares y Teléfonos". 

Este notebook guarda toda la información recolectada en el csv "df_total_original.csv".

# Índice

* [1 - Imports](#1)
* [2 - Recolección de datos](#2)

# 1 - Imports <a id="1"></a>

In [2]:
import numpy as np
import pandas as pd

import time
from time import process_time

from funciones import request_a_diccionario, generar_df

# 2 - Recolección de datos <a id="2"></a>

#### Analizamos los países que hay

In [3]:
lista_paises = request_a_diccionario("https://api.mercadolibre.com/sites")

In [4]:
lista_paises

[{'default_currency_id': 'USD', 'id': 'MSV', 'name': 'El Salvador'},
 {'default_currency_id': 'MXN', 'id': 'MLM', 'name': 'Mexico'},
 {'default_currency_id': 'PEN', 'id': 'MPE', 'name': 'Perú'},
 {'default_currency_id': 'BRL', 'id': 'MLB', 'name': 'Brasil'},
 {'default_currency_id': 'UYU', 'id': 'MLU', 'name': 'Uruguay'},
 {'default_currency_id': 'NIO', 'id': 'MNI', 'name': 'Nicaragua'},
 {'default_currency_id': 'DOP', 'id': 'MRD', 'name': 'Dominicana'},
 {'default_currency_id': 'GTQ', 'id': 'MGT', 'name': 'Guatemala'},
 {'default_currency_id': 'COP', 'id': 'MCO', 'name': 'Colombia'},
 {'default_currency_id': 'PYG', 'id': 'MPY', 'name': 'Paraguay'},
 {'default_currency_id': 'CLP', 'id': 'MLC', 'name': 'Chile'},
 {'default_currency_id': 'VES', 'id': 'MLV', 'name': 'Venezuela'},
 {'default_currency_id': 'CRC', 'id': 'MCR', 'name': 'Costa Rica'},
 {'default_currency_id': 'HNL', 'id': 'MHN', 'name': 'Honduras'},
 {'default_currency_id': 'USD', 'id': 'MEC', 'name': 'Ecuador'},
 {'default_cu

### NOTA: vamos a recolectar información sólo de Argentina.

#### Analizamos las categorías principales que hay

In [5]:
lista_categorias_principales = request_a_diccionario("https://api.mercadolibre.com/sites/MLA/categories")

In [6]:
lista_categorias_principales

[{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
 {'id': 'MLA1512', 'name': 'Agro'},
 {'id': 'MLA1403', 'name': 'Alimentos y Bebidas'},
 {'id': 'MLA1071', 'name': 'Animales y Mascotas'},
 {'id': 'MLA1367', 'name': 'Antigüedades y Colecciones'},
 {'id': 'MLA1368', 'name': 'Arte, Librería y Mercería'},
 {'id': 'MLA1743', 'name': 'Autos, Motos y Otros'},
 {'id': 'MLA1384', 'name': 'Bebés'},
 {'id': 'MLA1246', 'name': 'Belleza y Cuidado Personal'},
 {'id': 'MLA1039', 'name': 'Cámaras y Accesorios'},
 {'id': 'MLA1051', 'name': 'Celulares y Teléfonos'},
 {'id': 'MLA1648', 'name': 'Computación'},
 {'id': 'MLA1144', 'name': 'Consolas y Videojuegos'},
 {'id': 'MLA1276', 'name': 'Deportes y Fitness'},
 {'id': 'MLA5726', 'name': 'Electrodomésticos y Aires Ac.'},
 {'id': 'MLA1000', 'name': 'Electrónica, Audio y Video'},
 {'id': 'MLA2547', 'name': 'Entradas para Eventos'},
 {'id': 'MLA407134', 'name': 'Herramientas y Construcción'},
 {'id': 'MLA1574', 'name': 'Hogar, Muebles y Jardín'},
 {'

In [8]:
print("Hay un total de", len(lista_categorias_principales), "categorías principales.")

Hay un total de 31 categorías principales.


### NOTA: vamos a recolectar información sólo de la categoría "Celulares y Teléfonos"

#### Analizamos todas las subcategorías "hoja" de la categoría "Celulares y Teléfonos".

In [9]:
categorias_all = request_a_diccionario("https://api.mercadolibre.com/sites/MLA/categories/all")

In [10]:
lista_subcategorias_id = []
lista_subcategorias_name = []
lista_subcategorias_cant_items = []

for categoria_id in list(categorias_all.keys()):
    dicc_info_particular = categorias_all[categoria_id]
    categoria_name = dicc_info_particular["name"]
    cant_items_totales = dicc_info_particular["total_items_in_this_category"]
    lista_categorias_padre = dicc_info_particular['path_from_root']
    categoria_principal_id = lista_categorias_padre[0]["id"]
    if categoria_principal_id == "MLA1051" and len(lista_categorias_padre) > 1:
        lista_subcategorias_id.append(categoria_id)
        lista_subcategorias_name.append(categoria_name)
        lista_subcategorias_cant_items.append(cant_items_totales)

df_info_subcategorias = pd.DataFrame({"id": lista_subcategorias_id, "nombre":lista_subcategorias_name, 
              "cant_items":lista_subcategorias_cant_items})

In [11]:
df_info_subcategorias.sort_values("cant_items", ascending = False)

,id,nombre,cant_items
9,MLA3502,Accesorios para Celulares,204700
37,MLA432437,"Carcasas, Fundas y Protectores",140316
103,MLA5337,Fundas para Celulares,104235
7,MLA3813,Repuestos de Celulares,54556
0,MLA1055,Celulares y Smartphones,47157
105,MLA47540,Protectores de Pantalla,30985
45,MLA3517,Cargadores,18946
31,MLA10675,Displays y LCD,18732
8,MLA1053,Telefonía Fija e Inalámbrica,13395
106,MLA429749,Cargadores con Cable,13000


In [12]:
print("Hay un total de", len(df_info_subcategorias), "subcategorías 'hoja' de la categoría 'Celulares y Teléfonos'.")

Hay un total de 136 subcategorías 'hoja' de la categoría 'Celulares y Teléfonos'.


#### Recolectamos los datos usando la API

In [29]:
t_inicial = time.perf_counter()

lista_dfs = []
for numero_categoria, categoria_id in enumerate(lista_subcategorias_id):
    categoria_name = lista_subcategorias_name[numero_categoria]
    cant_items_totales = lista_subcategorias_cant_items[numero_categoria]
    print("Recolectando data de la categoría:", str(numero_categoria), "de", len(lista_subcategorias_id)-1, "(categoría", categoria_name, ").")
    limite_for = min(20, int(np.floor(cant_items_totales / 50)))
    for pagina in range(limite_for):
        print("Página:" + str(pagina) + " de 19.", end='\r', flush=True)
        #categoria_id = categoria["id"]
        #categoria_name = categoria["name"]
        offset = str(50 * pagina + 1)      
        try:
            items = request_a_diccionario("https://api.mercadolibre.com/sites/MLA/search?category=" + categoria_id + "&limit=50&offset=" + offset)
            df = generar_df(items)
            lista_dfs.append(df)
            time.sleep(1)
        except Exception as err:
            print("Hubo un error en la llamada de la página", pagina)
    elapsed_time = time.perf_counter() - t_inicial
    print("Tiempo parcial de ejecución:", round(elapsed_time / 60, 2), "minutos.")
    print("-"*100)
        
elapsed_time = time.perf_counter() - t_inicial

print("Tiempo total de ejecución:", round(elapsed_time / 60, 2), "minutos.")

Recolectando data de la categoría: 0 de 136 (categoría Celulares y Smartphones ).
Tiempo parcial de ejecución: 0.62 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 1 de 136 (categoría Lentes de Realidad Virtual ).
Tiempo parcial de ejecución: 1.15 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 2 de 136 (categoría Handies y Radiofrecuencia ).
Tiempo parcial de ejecución: 1.63 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 3 de 136 (categoría Tarifadores y Cabinas ).
Tiempo parcial de ejecución: 1.66 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 4 de 136 (categoría Smartwatches y Accesorios ).
Tiempo pa

Tiempo parcial de ejecución: 14.23 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 39 de 136 (categoría Chips GSM ).
Tiempo parcial de ejecución: 14.72 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 40 de 136 (categoría Protectores de Pantalla ).
Tiempo parcial de ejecución: 14.91 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 41 de 136 (categoría Lentes ).
Tiempo parcial de ejecución: 15.38 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 42 de 136 (categoría Otros ).
Tiempo parcial de ejecución: 15.38 minutos.
---------------------------------------------------------------------------------------------

Tiempo parcial de ejecución: 24.8 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 77 de 136 (categoría Pines Extractores de Sim ).
Tiempo parcial de ejecución: 24.8 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 78 de 136 (categoría Receptores Inalámbricos Qi ).
Tiempo parcial de ejecución: 24.82 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 79 de 136 (categoría Separadores de Displays ).
Tiempo parcial de ejecución: 24.82 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 80 de 136 (categoría Skins ).
Tiempo parcial de ejecución: 24.96 minutos.
------------------------------------------------------------

#### Juntamos todo en un sólo dataframe

In [30]:
df_total_original = pd.concat(lista_dfs, axis = 0)

#### Corroboramos el tamaño del dataset obtenido

In [34]:
print("Tamaño total:", len(df_total_original))

Tamaño total: 75313


#### Eliminamos duplicados

In [50]:
subset_columnas = list(df_total_original.columns)
subset_columnas.remove("tags")
print("Cantidad de duplicados:", df_total_original.duplicated(keep="first", subset = subset_columnas).sum())
df_total_original = df_total_original[~df_total_original.duplicated(keep="first", subset = subset_columnas)]
print("Eliminamos los duplicados.")
print("Quedaron", len(df_total_original), "registros en total.")

Cantidad de duplicados: 17367
Eliminamos los duplicados.
Quedaron 57946 registros en total.


#### Guardamos en un csv

In [51]:
df_total_original.to_csv("df_total_original.csv", index = False)

In [52]:
df_total_original.head(2)

,id,site_id,title,price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,condition,...,category_id,official_store_id,catalog_product_id,attributes__marca,attributes__modelo,seller__power_seller_status,seller__car_dealer,seller__eshop__eshop_experience,seller__eshop__eshop_id,tags
0,MLA856785349,MLA,Moto E6 Play 32 Gb Ocean Blue 2 Gb Ram,19990.0,ARS,50.0,5.0,buy_it_now,gold_special,new,...,MLA1055,NaN,MLA15243582,Motorola,E6 Play,silver,0.0,NaN,NaN,"[loyalty_discount_eligible, brand_verified, ex..."
1,MLA845554053,MLA,Moto G8 Play 32 Gb Knight Gray 2 Gb Ram,25999.0,ARS,1.0,200.0,buy_it_now,gold_special,new,...,MLA1055,NaN,MLA15273696,Motorola,G8 Play,platinum,0.0,0,162995,"[brand_verified, extended_warranty_eligible, g..."
